# Image Segementation


## Dependencies

In [ ]:
%pip install numpy pandas matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def load_grayscale_image(image_path):
    """Load image and convert to a grayscale matrix."""
    img = Image.open(image_path).convert('L')
    return np.array(img)

def show_image(matrix, title='Result'):
    """Display a matrix (image) using matplotlib."""
    plt.imshow(matrix, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Example usage pattern:
# 1. Load grayscale image.
# 2. Run your algorithm function on the matrix.
# 3. Show the result.

# Example:
# img = load_grayscale_image('your_image.jpg')
# result = your_segmentation_function(img)  # Replace with your algorithm
# show_image(result)


## Threshold based

In [ ]:
def global_threshold(img, T=127):
    return (img > T).astype(np.uint8) * 255


In [ ]:
def adaptive_threshold(img, window=15, C=5):
    pad = window // 2
    padded = np.pad(img, pad, mode='reflect')
    out = np.zeros_like(img)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            local = padded[i:i+window, j:j+window]
            T = np.mean(local) - C
            out[i,j] = 255 if img[i,j] > T else 0
    return out


In [ ]:
def otsu_threshold(img):
    hist, _ = np.histogram(img.flatten(), 256, [0,256])
    total = img.size
    sum_total = np.dot(hist, np.arange(256))
    
    sumB = 0
    wB = 0
    max_var = 0
    threshold = 0

    for t in range(256):
        wB += hist[t]
        if wB == 0: continue
        wF = total - wB
        if wF == 0: break

        sumB += t * hist[t]
        mB = sumB / wB
        mF = (sum_total - sumB) / wF

        var_between = wB * wF * (mB - mF)**2

        if var_between > max_var:
            max_var = var_between
            threshold = t

    return threshold

def otsu_segmentation(img):
    T = otsu_threshold(img)
    return global_threshold(img, T)


In [ ]:
def multilevel_threshold(img, t1=85, t2=170):
    out = np.zeros_like(img)
    out[img < t1] = 0
    out[(img >= t1) & (img < t2)] = 127
    out[img >= t2] = 255
    return out


In [ ]:
def histogram_threshold(img):
    hist, _ = np.histogram(img.flatten(), bins=256, range=(0,256))

    # Smooth histogram
    hist_smooth = np.convolve(hist, np.ones(5)/5, mode='same')

    peaks = np.argsort(hist_smooth)[-2:]  # top 2 peaks
    p1, p2 = sorted(peaks)

    valley = np.argmin(hist_smooth[p1:p2]) + p1
    return valley

def histogram_segmentation(img):
    T = histogram_threshold(img)
    return global_threshold(img, T)


In [ ]:
img = load_grayscale_image(r"C:\Users\SATYAKI MANDAL\OneDrive\Desktop\image-segmentation\images\geo-image.jpg")

result1 = global_threshold(img, 120)
result2 = adaptive_threshold(img)
result3 = otsu_segmentation(img)
result4 = multilevel_threshold(img)
result5 = histogram_segmentation(img)

show_image(result3, "Otsu Result")


## Edge based

In [ ]:
import numpy as np

def convolve(img, kernel):
    k = kernel.shape[0]
    pad = k // 2
    padded = np.pad(img, pad, mode='reflect')
    out = np.zeros_like(img, dtype=float)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            region = padded[i:i+k, j:j+k]
            out[i,j] = np.sum(region * kernel)
    return out


In [ ]:
def roberts_edge(img):
    kx = np.array([[1, 0],[0, -1]])
    ky = np.array([[0, 1],[-1, 0]])
    gx = convolve(img, kx)
    gy = convolve(img, ky)
    return np.sqrt(gx**2 + gy**2)


In [ ]:
def prewitt_edge(img):
    kx = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
    ky = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
    gx = convolve(img, kx)
    gy = convolve(img, ky)
    return np.sqrt(gx**2 + gy**2)


In [ ]:
def sobel_edge(img):
    kx = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    ky = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
    gx = convolve(img, kx)
    gy = convolve(img, ky)
    return np.sqrt(gx**2 + gy**2)


In [ ]:
def scharr_edge(img):
    kx = np.array([[3,0,-3],[10,0,-10],[3,0,-3]])
    ky = np.array([[3,10,3],[0,0,0],[-3,-10,-3]])
    gx = convolve(img, kx)
    gy = convolve(img, ky)
    return np.sqrt(gx**2 + gy**2)


In [ ]:
def laplacian_edge(img):
    kernel = np.array([[0,1,0],[1,-4,1],[0,1,0]])
    return np.abs(convolve(img, kernel))


In [ ]:
def gaussian_kernel(size=5, sigma=1):
    ax = np.linspace(-(size//2), size//2, size)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2)/(2*sigma**2))
    return kernel / np.sum(kernel)

def log_edge(img):
    smooth = convolve(img, gaussian_kernel(5,1))
    return laplacian_edge(smooth)


In [ ]:
def zero_crossing(img):
    zc = np.zeros_like(img)
    for i in range(1, img.shape[0]-1):
        for j in range(1, img.shape[1]-1):
            patch = img[i-1:i+2, j-1:j+2]
            if patch.max() > 0 and patch.min() < 0:
                zc[i,j] = 255
    return zc


In [ ]:
def canny_edge(img, low=50, high=100):
    smooth = convolve(img, gaussian_kernel(5,1))
    gx = convolve(smooth, np.array([[-1,0,1],[-2,0,2],[-1,0,1]]))
    gy = convolve(smooth, np.array([[1,2,1],[0,0,0],[-1,-2,-1]]))
    grad = np.sqrt(gx**2 + gy**2)

    strong = grad > high
    weak = (grad >= low) & (grad <= high)

    edges = np.zeros_like(img)
    edges[strong] = 255
    edges[weak] = 75
    return edges


In [ ]:
r1 = roberts_edge(img)
# r2 = sobel_edge(img)
# r3 = laplacian_edge(img)
# r4 = log_edge(img)
# r5 = zero_crossing(r4)
# r6 = canny_edge(img)

show_image(r1, "roberts_edge")
#show_image(r6, "Canny")


## Region based

In [ ]:
def is_similar(a, b, threshold):
    return abs(int(a) - int(b)) <= threshold

In [ ]:
def region_growing(img, seed=(100,100), threshold=10):
    h, w = img.shape
    segmented = np.zeros_like(img)
    stack = [seed]
    seed_val = img[seed]

    while stack:
        x, y = stack.pop()
        if segmented[x,y] == 1:
            continue
        segmented[x,y] = 255

        for dx in [-1,0,1]:
            for dy in [-1,0,1]:
                nx, ny = x+dx, y+dy
                if 0 <= nx < h and 0 <= ny < w:
                    if segmented[nx,ny] == 0 and is_similar(img[nx,ny], seed_val, threshold):
                        stack.append((nx,ny))
    return segmented


In [ ]:
def seeded_region_growing(img, seeds, threshold=10):
    h, w = img.shape
    segmented = np.zeros_like(img)
    
    for label, seed in enumerate(seeds, start=1):
        stack = [seed]
        seed_val = img[seed]

        while stack:
            x, y = stack.pop()
            if segmented[x,y] != 0:
                continue
            segmented[x,y] = label * 40  # different gray for each region

            for dx in [-1,0,1]:
                for dy in [-1,0,1]:
                    nx, ny = x+dx, y+dy
                    if 0 <= nx < h and 0 <= ny < w:
                        if segmented[nx,ny] == 0 and is_similar(img[nx,ny], seed_val, threshold):
                            stack.append((nx,ny))
    return segmented


In [ ]:
def split_region(img, x, y, size, threshold, out):
    region = img[x:x+size, y:y+size]
    if np.var(region) < threshold or size <= 4:
        out[x:x+size, y:y+size] = np.mean(region)
        return

    half = size // 2
    split_region(img, x, y, half, threshold, out)
    split_region(img, x+half, y, half, threshold, out)
    split_region(img, x, y+half, half, threshold, out)
    split_region(img, x+half, y+half, half, threshold, out)

def region_splitting(img, threshold=100):
    out = np.zeros_like(img)
    split_region(img, 0, 0, min(img.shape), threshold, out)
    return out


In [ ]:
def region_merging(img, threshold=10):
    h, w = img.shape
    merged = img.copy()

    for i in range(h-1):
        for j in range(w-1):
            if is_similar(merged[i,j], merged[i,j+1], threshold):
                merged[i,j+1] = merged[i,j]
            if is_similar(merged[i,j], merged[i+1,j], threshold):
                merged[i+1,j] = merged[i,j]
    return merged


In [ ]:
def split_and_merge(img, split_thresh=100, merge_thresh=10):
    split = region_splitting(img, split_thresh)
    merged = region_merging(split, merge_thresh)
    return merged


In [ ]:
r1 = region_growing(img, seed=(120,150))
# r2 = seeded_region_growing(img, [(50,50),(150,150)])
# r3 = region_splitting(img)
# r4 = region_merging(img)
# r5 = split_and_merge(img)

show_image(r1, "Region Growing")
# show_image(r5, "Split & Merge")

## Clustering based

In [ ]:
def kmeans_segmentation(img, k=3, iters=10):
    pixels = img.reshape(-1,1).astype(float)
    centers = np.random.choice(pixels.flatten(), k)

    for _ in range(iters):
        distances = np.abs(pixels - centers.reshape(1,-1))
        labels = np.argmin(distances, axis=1)

        for i in range(k):
            centers[i] = pixels[labels==i].mean()

    return labels.reshape(img.shape) * (255//k)


In [ ]:
def fcm_segmentation(img, c=3, m=2, iters=10):
    pixels = img.flatten().astype(float)
    n = len(pixels)
    U = np.random.dirichlet(np.ones(c), size=n)

    centers = np.zeros(c)

    for _ in range(iters):
        for j in range(c):
            centers[j] = np.sum((U[:,j]**m) * pixels) / np.sum(U[:,j]**m)

        for i in range(n):
            for j in range(c):
                denom = sum((abs(pixels[i]-centers[j]) /
                             abs(pixels[i]-centers[k]))**(2/(m-1))
                            for k in range(c))
                U[i,j] = 1/denom

    labels = np.argmax(U, axis=1)
    return labels.reshape(img.shape) * (255//c)


In [ ]:
def mean_shift_segmentation(img, radius=20, iters=5):
    pixels = img.flatten().astype(float)
    shifted = pixels.copy()

    for _ in range(iters):
        for i in range(len(pixels)):
            neighbors = pixels[np.abs(pixels - shifted[i]) < radius]
            shifted[i] = neighbors.mean()

    unique_vals = np.unique(shifted.round())
    label_map = {v:i for i,v in enumerate(unique_vals)}
    labels = np.array([label_map[v] for v in shifted.round()])

    return labels.reshape(img.shape) * (255//len(unique_vals))


In [ ]:
def hierarchical_segmentation(img, clusters=3):
    pixels = img.flatten().astype(float)
    groups = [[p] for p in pixels]

    while len(groups) > clusters:
        min_dist = float('inf')
        merge_idx = (0,1)

        for i in range(len(groups)):
            for j in range(i+1, len(groups)):
                dist = abs(np.mean(groups[i]) - np.mean(groups[j]))
                if dist < min_dist:
                    min_dist = dist
                    merge_idx = (i,j)

        g1, g2 = merge_idx
        groups[g1] += groups[g2]
        groups.pop(g2)

    label_vals = {}
    for idx, g in enumerate(groups):
        for val in g:
            label_vals[val] = idx

    labels = np.array([label_vals[p] for p in pixels])
    return labels.reshape(img.shape) * (255//clusters)


In [ ]:
def gmm_segmentation(img, k=3, iters=10):
    pixels = img.flatten().astype(float)
    n = len(pixels)

    means = np.random.choice(pixels, k)
    variances = np.ones(k) * 100
    weights = np.ones(k) / k

    for _ in range(iters):
        # E-step
        probs = np.zeros((n,k))
        for j in range(k):
            probs[:,j] = weights[j] * np.exp(-(pixels-means[j])**2 / (2*variances[j]))

        probs /= probs.sum(axis=1, keepdims=True)

        # M-step
        for j in range(k):
            w = probs[:,j].sum()
            means[j] = np.sum(probs[:,j]*pixels) / w
            variances[j] = np.sum(probs[:,j]*(pixels-means[j])**2) / w
            weights[j] = w / n

    labels = np.argmax(probs, axis=1)
    return labels.reshape(img.shape) * (255//k)


In [ ]:
r1 = kmeans_segmentation(img, k=3)
# r2 = fcm_segmentation(img)
# r3 = mean_shift_segmentation(img)
# r4 = hierarchical_segmentation(img)
# r5 = gmm_segmentation(img)

show_image(r1, "K-Means")
# show_image(r5, "GMM")

## Graph based

In [ ]:
def build_graph(img):
    h, w = img.shape
    edges = []

    for i in range(h):
        for j in range(w):
            idx = i*w + j
            if j+1 < w:
                right = i*w + (j+1)
                weight = abs(int(img[i,j]) - int(img[i,j+1]))
                edges.append((idx, right, weight))
            if i+1 < h:
                down = (i+1)*w + j
                weight = abs(int(img[i,j]) - int(img[i+1,j]))
                edges.append((idx, down, weight))
    return edges, h, w


In [ ]:
def mst_segmentation(img, threshold=10):
    edges, h, w = build_graph(img)
    parent = list(range(h*w))

    def find(x):
        while parent[x] != x:
            x = parent[x]
        return x

    def union(x,y):
        parent[find(x)] = find(y)

    edges.sort(key=lambda x: x[2])

    for u,v,wgt in edges:
        if wgt < threshold:
            union(u,v)

    labels = np.array([find(i) for i in range(h*w)])
    return labels.reshape(h,w) % 255


In [ ]:
def normalized_cut_segmentation(img, threshold=15):
    edges, h, w = build_graph(img)
    labels = np.zeros(h*w)

    for u,v,wgt in edges:
        if wgt < threshold:
            labels[v] = labels[u]

    return labels.reshape(h,w) * 20


In [ ]:
def graph_cut_segmentation(img, threshold=20):
    edges, h, w = build_graph(img)
    labels = np.zeros(h*w)

    for u,v,wgt in edges:
        if wgt < threshold:
            labels[v] = labels[u]
        else:
            labels[v] = 255

    return labels.reshape(h,w)


In [ ]:
def random_walker_segmentation(img, seed=(100,100), threshold=10):
    h, w = img.shape
    labels = np.zeros((h,w))
    stack = [seed]
    seed_val = img[seed]

    while stack:
        x,y = stack.pop()
        labels[x,y] = 255

        for dx,dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx,ny = x+dx, y+dy
            if 0<=nx<h and 0<=ny<w:
                if labels[nx,ny]==0 and abs(int(img[nx,ny]) - int(seed_val)) < threshold:
                    stack.append((nx,ny))
    return labels


In [ ]:
r1 = mst_segmentation(img)
# r2 = normalized_cut_segmentation(img)
# r3 = graph_cut_segmentation(img)
# r4 = random_walker_segmentation(img, seed=(120,120))

show_image(r1, "MST")
# show_image(r4, "Random Walker")


## Watershed Segmentation Based

In [ ]:
def sobel_gradient(img):
    kx = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    ky = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
    gx = convolve(img, kx)
    gy = convolve(img, ky)
    return np.sqrt(gx**2 + gy**2)


In [ ]:
def marker_watershed(img, markers, threshold=10):
    grad = sobel_gradient(img)
    h, w = img.shape
    labels = np.zeros((h,w))
    stack = []

    # Initialize markers
    for label, (x,y) in enumerate(markers, start=1):
        labels[x,y] = label
        stack.append((x,y,label))

    while stack:
        x,y,label = stack.pop(0)

        for dx,dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx,ny = x+dx, y+dy
            if 0<=nx<h and 0<=ny<w and labels[nx,ny]==0:
                if grad[nx,ny] < threshold:
                    labels[nx,ny] = label
                    stack.append((nx,ny,label))

    return labels * 40


In [ ]:
def distance_transform(binary):
    h, w = binary.shape
    dist = np.full((h,w), np.inf)

    for i in range(h):
        for j in range(w):
            if binary[i,j] == 0:
                dist[i,j] = 0

    for i in range(h):
        for j in range(w):
            if i>0: dist[i,j] = min(dist[i,j], dist[i-1,j]+1)
            if j>0: dist[i,j] = min(dist[i,j], dist[i,j-1]+1)

    for i in reversed(range(h)):
        for j in reversed(range(w)):
            if i<h-1: dist[i,j] = min(dist[i,j], dist[i+1,j]+1)
            if j<w-1: dist[i,j] = min(dist[i,j], dist[i,j+1]+1)

    return dist


In [ ]:
def distance_watershed(img, thresh=127):
    binary = (img > thresh).astype(np.uint8)
    dist = distance_transform(binary)

    markers = np.zeros_like(img)
    label = 1

    for i in range(1,img.shape[0]-1):
        for j in range(1,img.shape[1]-1):
            if dist[i,j] == np.max(dist[i-1:i+2,j-1:j+2]):
                markers[i,j] = label
                label += 1

    return marker_watershed(img, [(i,j) for i,j in zip(*np.where(markers>0))])


In [ ]:
r1 = marker_watershed(img, markers=[(50,50),(150,150)])
# r2 = distance_watershed(img)

show_image(r1, "Marker Watershed")
# show_image(r2, "Distance Watershed")


## Transform-Based Segmentation

In [ ]:
def hough_lines(edge_img, rho_res=1, theta_res=1):
    h, w = edge_img.shape
    diag = int(np.sqrt(h*h + w*w))
    rhos = np.arange(-diag, diag, rho_res)
    thetas = np.deg2rad(np.arange(-90, 90, theta_res))

    acc = np.zeros((len(rhos), len(thetas)))

    ys, xs = np.where(edge_img)
    for x, y in zip(xs, ys):
        for t_idx, theta in enumerate(thetas):
            rho = int(x*np.cos(theta) + y*np.sin(theta)) + diag
            acc[rho, t_idx] += 1

    return acc, rhos, thetas


In [ ]:
def hough_circles(edge_img, radius):
    h, w = edge_img.shape
    acc = np.zeros((h,w))

    ys, xs = np.where(edge_img)
    for x, y in zip(xs, ys):
        for theta in range(0,360,10):
            a = int(x - radius*np.cos(np.deg2rad(theta)))
            b = int(y - radius*np.sin(np.deg2rad(theta)))
            if 0 <= a < w and 0 <= b < h:
                acc[b,a] += 1

    return acc


In [ ]:
def radon_transform(img, angles=np.arange(0,180,5)):
    h, w = img.shape
    center = (h//2, w//2)
    sinogram = []

    for theta in np.deg2rad(angles):
        proj = []
        for x in range(h):
            sum_val = 0
            for y in range(w):
                xp = int((x-center[0])*np.cos(theta) + (y-center[1])*np.sin(theta) + center[0])
                if 0 <= xp < h:
                    sum_val += img[xp,y]
            proj.append(sum_val)
        sinogram.append(proj)

    return np.array(sinogram)


In [ ]:
edges = sobel_edge(img)
edges = (edges > 100).astype(np.uint8)  # binary edge map

acc_lines, rhos, thetas = hough_lines(edges)
# acc_circles = hough_circles(edges, radius=30)
# radon_img = radon_transform(img)

show_image(acc_lines, "Hough Lines Accumulator")
# show_image(acc_circles, "Circle Hough")
# show_image(radon_img, "Radon Transform")


## Texture-Based Segmentation

In [ ]:
def glcm_matrix(img, levels=16, dx=1, dy=0):
    img_q = (img / (256//levels)).astype(int)
    glcm = np.zeros((levels, levels))

    for i in range(img.shape[0]-dy):
        for j in range(img.shape[1]-dx):
            p = img_q[i,j]
            q = img_q[i+dy,j+dx]
            glcm[p,q] += 1

    return glcm / glcm.sum()

def glcm_contrast(glcm):
    levels = glcm.shape[0]
    contrast = 0
    for i in range(levels):
        for j in range(levels):
            contrast += glcm[i,j] * (i-j)**2
    return contrast


In [ ]:
def lbp_image(img):
    h, w = img.shape
    out = np.zeros((h-2, w-2))

    for i in range(1,h-1):
        for j in range(1,w-1):
            center = img[i,j]
            code = 0
            idx = 0
            for dx in [-1,0,1]:
                for dy in [-1,0,1]:
                    if dx==0 and dy==0: continue
                    code |= (img[i+dx,j+dy] > center) << idx
                    idx += 1
            out[i-1,j-1] = code
    return out


In [ ]:
def gabor_kernel(size=21, sigma=5, theta=0, freq=0.1):
    ax = np.linspace(-size//2, size//2, size)
    xx, yy = np.meshgrid(ax, ax)

    x_theta = xx*np.cos(theta) + yy*np.sin(theta)
    kernel = np.exp(-(xx**2+yy**2)/(2*sigma**2)) * np.cos(2*np.pi*freq*x_theta)
    return kernel

def gabor_filter(img):
    kernel = gabor_kernel()
    return np.abs(convolve(img, kernel))

In [ ]:
def laws_masks():
    L = np.array([1,4,6,4,1])
    E = np.array([-1,-2,0,2,1])
    S = np.array([-1,0,2,0,-1])
    return [np.outer(L,E), np.outer(E,S), np.outer(S,L)]

def laws_texture(img):
    masks = laws_masks()
    energy = np.zeros_like(img, dtype=float)

    for m in masks:
        filtered = convolve(img, m)
        energy += np.abs(filtered)

    return energy


In [ ]:
glcm = glcm_matrix(img)
contrast = glcm_contrast(glcm)

lbp_map = lbp_image(img)
# gabor_map = gabor_filter(img)
# laws_map = laws_texture(img)

show_image(lbp_map, "LBP")
# show_image(gabor_map, "Gabor")
# show_image(laws_map, "Laws Texture")


## Model-Based Segmentation


In [ ]:
def active_contour(img, center=(100,100), radius=50, alpha=0.1, beta=0.1, iters=100):
    t = np.linspace(0, 2*np.pi, 100)
    x = center[0] + radius*np.cos(t)
    y = center[1] + radius*np.sin(t)

    grad = sobel_edge(img)

    for _ in range(iters):
        for i in range(len(x)):
            xi, yi = int(x[i]), int(y[i])

            # image force (toward edges)
            fx = grad[min(xi+1,img.shape[0]-1), yi] - grad[max(xi-1,0), yi]
            fy = grad[xi, min(yi+1,img.shape[1]-1)] - grad[xi, max(yi-1,0)]

            # smoothness
            x_prev, x_next = x[i-1], x[(i+1)%len(x)]
            y_prev, y_next = y[i-1], y[(i+1)%len(y)]

            x[i] += alpha*(x_prev + x_next - 2*x[i]) + beta*fx
            y[i] += alpha*(y_prev + y_next - 2*y[i]) + beta*fy

    mask = np.zeros_like(img)
    mask[x.astype(int), y.astype(int)] = 255
    return mask


In [ ]:
def level_set_segmentation(img, iters=50):
    phi = np.ones_like(img, dtype=float)
    phi[50:-50,50:-50] = -1  # initial contour

    grad = sobel_edge(img)

    for _ in range(iters):
        phi_x = np.gradient(phi, axis=0)
        phi_y = np.gradient(phi, axis=1)
        mag = np.sqrt(phi_x**2 + phi_y**2)

        F = -grad
        phi += F * mag * 0.1

    return (phi <= 0).astype(np.uint8) * 255


In [ ]:
def shape_model_segmentation(img, template_center=(100,100), radius=40):
    h, w = img.shape
    mask = np.zeros((h,w))

    for i in range(h):
        for j in range(w):
            if (i-template_center[0])**2 + (j-template_center[1])**2 < radius**2:
                mask[i,j] = 255
    return mask


In [ ]:
# snake = active_contour(img)
# ls = level_set_segmentation(img)
# shape = shape_model_segmentation(img)

# show_image(snake, "Active Contour")
# show_image(ls, "Level Set")
# show_image(shape, "Shape Model")


## Machine Learning-Based Segmentation

In [ ]:
def prepare_data(img, threshold=128):
    X = img.flatten().astype(float).reshape(-1,1)
    y = (img.flatten() > threshold).astype(int)  # pseudo labels
    return X, y

In [ ]:
def svm_segmentation(img, lr=0.001, C=1, iters=200):
    X, y = prepare_data(img)
    y = np.where(y==0, -1, 1)

    w = np.zeros(1)
    b = 0

    for _ in range(iters):
        for xi, yi in zip(X, y):
            if yi*(np.dot(xi,w)+b) < 1:
                w += lr*(yi*xi - 2*C*w)
                b += lr*yi
            else:
                w += lr*(-2*C*w)

    preds = np.sign(X.dot(w) + b)
    return preds.reshape(img.shape) * 255


In [ ]:
def decision_stump(X, y):
    thresh = np.mean(X)
    left = y[X[:,0] <= thresh]
    right = y[X[:,0] > thresh]
    return thresh, int(np.mean(left)>0.5), int(np.mean(right)>0.5)

def random_forest_segmentation(img, trees=5):
    X, y = prepare_data(img)
    forest = []

    for _ in range(trees):
        idx = np.random.choice(len(X), len(X)//2)
        stump = decision_stump(X[idx], y[idx])
        forest.append(stump)

    preds = []
    for xi in X:
        votes = []
        for thresh, l, r in forest:
            votes.append(l if xi<=thresh else r)
        preds.append(int(np.mean(votes)>0.5))

    return np.array(preds).reshape(img.shape)*255


In [ ]:
def naive_bayes_segmentation(img):
    X, y = prepare_data(img)

    X0 = X[y==0]
    X1 = X[y==1]

    mu0, mu1 = X0.mean(), X1.mean()
    var0, var1 = X0.var(), X1.var()
    p0, p1 = len(X0)/len(X), len(X1)/len(X)

    def gaussian(x, mu, var):
        return np.exp(-(x-mu)**2/(2*var)) / np.sqrt(2*np.pi*var)

    probs0 = gaussian(X, mu0, var0) * p0
    probs1 = gaussian(X, mu1, var1) * p1

    preds = (probs1 > probs0).astype(int)
    return preds.reshape(img.shape)*255


In [ ]:
# svm_map = svm_segmentation(img)
# rf_map = random_forest_segmentation(img)
# nb_map = naive_bayes_segmentation(img)

# show_image(svm_map, "SVM")
# show_image(rf_map, "Random Forest")
# show_image(nb_map, "Naive Bayes")


## Deep Learning-Based Segmentation

In [ ]:
def conv2d(img, kernel):
    k = kernel.shape[0]
    pad = k // 2
    padded = np.pad(img, pad, mode='reflect')
    out = np.zeros_like(img, dtype=float)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            region = padded[i:i+k, j:j+k]
            out[i,j] = np.sum(region * kernel)
    return out


In [ ]:
def relu(x):
    return np.maximum(0, x)


In [ ]:
def max_pool(img, size=2):
    h, w = img.shape
    out = np.zeros((h//size, w//size))
    for i in range(0,h,size):
        for j in range(0,w,size):
            out[i//size,j//size] = np.max(img[i:i+size,j:j+size])
    return out


In [ ]:
def upsample(img, scale=2):
    return np.repeat(np.repeat(img, scale, axis=0), scale, axis=1)


In [ ]:
def fcn_segmentation(img):
    k = np.ones((3,3))/9
    x = relu(conv2d(img, k))
    x = max_pool(x)
    x = relu(conv2d(x, k))
    x = upsample(x)
    return (x > x.mean()).astype(np.uint8)*255


In [ ]:
def unet_segmentation(img):
    k = np.ones((3,3))/9

    enc1 = relu(conv2d(img, k))
    p1 = max_pool(enc1)

    enc2 = relu(conv2d(p1, k))
    p2 = max_pool(enc2)

    bottleneck = relu(conv2d(p2, k))

    up1 = upsample(bottleneck)
    dec1 = relu(conv2d(up1 + enc2, k))

    up2 = upsample(dec1)
    dec2 = relu(conv2d(up2 + enc1, k))

    return (dec2 > dec2.mean()).astype(np.uint8)*255


In [ ]:
def segnet_segmentation(img):
    k = np.ones((3,3))/9

    x = relu(conv2d(img, k))
    pooled = max_pool(x)
    up = upsample(pooled)
    x = relu(conv2d(up, k))

    return (x > x.mean()).astype(np.uint8)*255


In [ ]:
def atrous_conv(img, kernel, rate=2):
    k = kernel.shape[0]
    pad = k//2 * rate
    padded = np.pad(img, pad, mode='reflect')
    out = np.zeros_like(img)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            region = padded[i:i+k*rate:rate, j:j+k*rate:rate]
            out[i,j] = np.sum(region * kernel)
    return out

def deeplab_segmentation(img):
    k = np.ones((3,3))/9
    x = relu(atrous_conv(img, k, rate=2))
    x = relu(atrous_conv(x, k, rate=4))
    return (x > x.mean()).astype(np.uint8)*255


In [ ]:
def mask_rcnn_segmentation(img):
    edges = sobel_edge(img)
    mask = (edges > edges.mean()).astype(np.uint8)*255
    return mask


In [ ]:
# fcn = fcn_segmentation(img)
# unet = unet_segmentation(img)
# segnet = segnet_segmentation(img)
# deeplab = deeplab_segmentation(img)
# maskrcnn = mask_rcnn_segmentation(img)

# show_image(unet, "U-Net")
# show_image(deeplab, "DeepLab")

## Hybrid Segmentation Techniques

In [ ]:
def edge_region_segmentation(img, seed=(100,100), thresh=10):
    edges = sobel_edge(img)
    edges = (edges > edges.mean()).astype(np.uint8)

    h, w = img.shape
    seg = np.zeros_like(img)
    stack = [seed]
    seed_val = img[seed]

    while stack:
        x,y = stack.pop()
        if seg[x,y] == 255 or edges[x,y] == 1:
            continue

        seg[x,y] = 255

        for dx,dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx,ny = x+dx, y+dy
            if 0<=nx<h and 0<=ny<w:
                if abs(int(img[nx,ny])-int(seed_val)) < thresh:
                    stack.append((nx,ny))
    return seg


In [ ]:
def threshold_clustering(img):
    binary = global_threshold(img, 127)
    clustered = kmeans_segmentation(binary, k=2)
    return clustered


In [ ]:
def crf_refinement(img, mask, thresh=10):
    refined = mask.copy()
    h, w = img.shape

    for i in range(1,h-1):
        for j in range(1,w-1):
            neighbors = img[i-1:i+2,j-1:j+2]
            if np.var(neighbors) < thresh:
                refined[i,j] = np.mean(mask[i-1:i+2,j-1:j+2])
    return refined

def cnn_crf_segmentation(img):
    cnn_mask = unet_segmentation(img)
    return crf_refinement(img, cnn_mask)


In [ ]:
def erode(img, size=3):
    pad = size//2
    padded = np.pad(img, pad, mode='constant')
    out = np.zeros_like(img)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            out[i,j] = np.min(padded[i:i+size, j:j+size])
    return out

def dilate(img, size=3):
    pad = size//2
    padded = np.pad(img, pad, mode='constant')
    out = np.zeros_like(img)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            out[i,j] = np.max(padded[i:i+size, j:j+size])
    return out

def morphology_watershed(img):
    binary = global_threshold(img, 127)
    clean = dilate(erode(binary))
    return distance_watershed(clean)


In [ ]:
# r1 = edge_region_segmentation(img)
# r2 = threshold_clustering(img)
# r3 = cnn_crf_segmentation(img)
# r4 = morphology_watershed(img)

# show_image(r1, "Edge + Region")
# show_image(r3, "CNN + CRF")
